This notebook is designed to alter webnucleo xml files by removing any and all data corresponding to a particular species. The notebook requires wnutils, but will quietly handle the installation in the 1st cell. 

In [1]:
import sys
!{sys.executable} -m pip install --quiet wnutils 
!{sys.executable} -m pip install --quiet requests

import wnutils.xml as wx
import matplotlib.pyplot as plt
import numpy as np
import os, io, requests

The cell below contains the function that will remove all the data pertaining to the list of species you feed it and write the new xml.

In [2]:
def remove_species(xml, species_to_remove):
    my_type = xml.get_type()
    new_xml = wx.New_Xml(my_type)
    
    #remove nuclides
    if my_type is not 'zone_data':

        nuclides = xml.get_nuclide_data()

        for s in species_to_remove:
            if s in nuclides:
                nuclides.pop(s)
        
        new_xml.set_nuclide_data(nuclides)
        
    #remove reactions
    if type is not 'zone_data':

        reactions = xml.get_reaction_data()

        reac_xpath = '[(reactant = \'' + species_to_remove[0] + \
            '\' or product = \'' + species_to_remove[0] + '\')'
        
        for i in range(1, len(species_to_remove)):
            reac_xpath += ' or (reactant = \'' + species_to_remove[i] +\
                '\' or product = \'' + species_to_remove[i] + '\')'
        
        reac_xpath += ']'

        reactions_to_remove = xml.get_reaction_data(reac_xpath)

        for s in reactions_to_remove:
            if s in reactions:
                reactions.pop(s)
        new_xml.set_reaction_data(reactions)

    #remove nuclides from zone data
    if type == 'zone_data' or type == 'libnucnet_input':
        zones = xml.get_zone_data()

        t_list =[]
        for s in species_to_remove:
            z, a, state = xml.get_z_a_state_from_nuclide_name(s)
            t_list.append((s,z,a))
        
        for zone in zones:
            for t in t_list:
                if t in zones[zone]['mass fractions']:
                    zones[zone]['mass fractions'].pop(t)
        
        new_xml.set_zone_data(zones)
    
    return new_xml

Begin by reading in the old XML. You may read the data over the web from an appropriate URL (the default for this notebook downloads the Webnucleo V2.2 Reaclib data from [OSF](https://osf.io/kyhbs/) or from a local directory (as in the comment). If you are running the notebook on Google Colab, you can upload the data to the work directory by clicking on the directory tab to the left and then clicking the upload symbol. On Colab, you may also read the file from Google Drive (click the Drive icon and follow the instructions).

In [3]:
xml = wx.Xml(io.BytesIO(requests.get('https://osf.io/kyhbs/download').content))
#xml= wx.Xml("example.xml")

Choose the species to remove.

In [4]:
species_to_remove = ['kr84','kr85','kr86']

This cell creates a new xml object of the same type as the input xml but with the specified species removed.  This operation can take some time.

In [ ]:
new_xml = remove_species(xml, species_to_remove)

Output the new file.  Change the name of the output file to desired string.

In [ ]:
new_file_name = "my_output.xml"
new_xml.write(new_file_name)